In [6]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from torch.autograd import Variable
import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torch import nn,optim
from torch.nn import functional as F 
from torch.utils.data import Dataset,DataLoader,TensorDataset
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [7]:
traindata = np.load('2ddata.npy')
trainlabel = np.load('2dlabel.npy')

In [8]:
trainlabel

array([ 43.52606322, -13.99713994,   9.49569717, ..., -12.22158433,
         6.01721102, -24.02406788])

In [9]:
X_train, X_test, Y_train, Y_test = train_test_split(traindata, trainlabel, test_size = 0.2, random_state = 1)
X_train, Y_train = torch.FloatTensor(X_train), torch.FloatTensor(Y_train)
X_test, Y_test = torch.FloatTensor(X_test), torch.FloatTensor(Y_test)

In [10]:
batchsize = 8
train_dataset = torch.utils.data.TensorDataset(X_train, Y_train)
test_dataset = torch.utils.data.TensorDataset(X_test, Y_test)

#dataloader 
train_loader = torch.utils.data.DataLoader(dataset = train_dataset, batch_size = batchsize, shuffle = True)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset, batch_size = batchsize, shuffle = False)

In [11]:
X_train.shape 

torch.Size([16000, 32, 32])

In [12]:
class CNN_16d2(nn.Module):
    def __init__(self):
        super(CNN_16d2,self).__init__()
        self.conv1 = nn.Conv2d(1, 4, 2,  stride=2)
        self.conv2 = nn.Conv2d(4, 16, 2, stride=2)
        self.conv3 = nn.Conv2d(16, 160, 2, stride=2)
        self.conv4 = nn.Conv2d(160, 40, 2, stride=2)
        self.conv5 = nn.Conv2d(40, 10, 2, stride=2)

        self.flat = nn.Flatten()
        self.fc1 = nn.Linear(10, 60)
        self.fc2 = nn.Linear(60, 60)
        self.fc3 = nn.Linear(60, 20)
        self.fc4 = nn.Linear(20, 1)
    
    def forward(self,x):
        x = self.conv1(x)
        #print(x.shape)
        x = self.conv2(x)
        #print(x.shape)
        x = self.conv3(x)
        #print(x.shape)
        x = self.conv4(x)  
        #print(x.shape)
        x = self.conv5(x)  
        #print(x.shape)

        x = self.flat(x)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [29]:
X = torch.randn(1,1,32,32)
xiao = CNN_16d2()
xiao(X)

tensor([[-0.2673]], grad_fn=<AddmmBackward0>)

In [30]:
model = CNN_16d2()
if torch.cuda.is_available():
    print("gpu_available")
    model.cuda()

gpu_available


In [31]:
optimizer = optim.Adam(model.parameters(), lr =0.001)
criteon= nn.MSELoss()
EPOCH = 5
loss_list = []
for epoch in range(EPOCH):
    model.train()
    sum_loss = 0 
    for batch_idx, (data, target) in enumerate(train_loader):
        if torch.cuda.is_available():
            data, target = data.cuda(), target.cuda()
        #data, target = Variable(data), Variable(target)

        data = torch.reshape(data,(batchsize, 1, 32,32))
        target = torch.reshape(target, (batchsize, 1))
        optimizer.zero_grad()
        
        
        data=data.to(torch.float32)
        target=target.to(torch.float32)
        output = model(data)

        
        #print(data)
        #print(target)
        #print("The output is:")
        #print(output)
        
        loss = criteon(output, target)
        loss.backward()
        optimizer.step()

        sum_loss += loss.item()
        if batch_idx % 10 == 9:
            print("train epoch {} loss : {:.3f} ".format(epoch, sum_loss/10))
            loss_list.append(sum_loss/100)
            sum_loss = 0      

train epoch 0 loss : 1754.225 
train epoch 0 loss : 1439.253 
train epoch 0 loss : 1851.075 
train epoch 0 loss : 1535.793 
train epoch 0 loss : 1619.769 
train epoch 0 loss : 1710.460 
train epoch 0 loss : 2180.364 
train epoch 0 loss : 1733.705 
train epoch 0 loss : 842.527 
train epoch 0 loss : 2245.000 
train epoch 0 loss : 1624.936 
train epoch 0 loss : 1783.496 
train epoch 0 loss : 1345.624 
train epoch 0 loss : 1216.751 
train epoch 0 loss : 1257.455 
train epoch 0 loss : 1345.824 
train epoch 0 loss : 1869.114 
train epoch 0 loss : 1772.347 
train epoch 0 loss : 1557.747 
train epoch 0 loss : 2129.412 
train epoch 0 loss : 1693.431 
train epoch 0 loss : 1510.720 
train epoch 0 loss : 1661.569 
train epoch 0 loss : 1704.308 
train epoch 0 loss : 1928.299 
train epoch 0 loss : 1367.831 
train epoch 0 loss : 1706.397 
train epoch 0 loss : 1347.812 
train epoch 0 loss : 1137.558 
train epoch 0 loss : 1822.982 
train epoch 0 loss : 1901.612 
train epoch 0 loss : 961.448 
train epoc

In [32]:
model = model.cpu()
re = model(torch.Tensor(X_test.reshape(4000,1,32,32))).detach().numpy()
re = re.reshape(4000)
re

array([ 74.59051  ,   6.0666823,  53.21632  , ...,   0.5124746,
       -12.01869  ,  -2.283545 ], dtype=float32)

In [33]:
Y_test

tensor([ 61.5076,   4.2478,  48.3385,  ...,   3.4108, -27.6324,  -7.8302])

In [36]:
resultvalue = pd.DataFrame([re,Y_test.numpy()]).T
resultvalue.columns = ['pre','true']
resultvalue

,pre,true
0,74.590508,61.507622
1,6.066682,4.247818
2,53.216320,48.338482
3,38.949986,23.041401
4,75.976379,68.316994
...,...,...
3995,-1.680231,-6.869918
3996,1.164764,4.799513
3997,0.512475,3.410830
3998,-12.018690,-27.632389


In [37]:
resultvalue['mape'] = np.abs(resultvalue['pre'] - resultvalue['true'])/np.abs(resultvalue['true'])
resultvalue

,pre,true,mape
0,74.590508,61.507622,0.212703
1,6.066682,4.247818,0.428188
2,53.216320,48.338482,0.100910
3,38.949986,23.041401,0.690435
4,75.976379,68.316994,0.112115
...,...,...,...
3995,-1.680231,-6.869918,0.755422
3996,1.164764,4.799513,0.757316
3997,0.512475,3.410830,0.849751
3998,-12.018690,-27.632389,0.565051


In [41]:
resultvalue['mape'].median()

0.21775773173050655

In [40]:
resultvalue.to_csv("hhh")